# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = False

In [4]:
PREDICTION_METHOD = "Module-based"

In [5]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses')

In [6]:
# OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
# display(OUTPUT_DATA_DIR)
# OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
INPUT_DATA_DIR = Path(
    OUTPUT_DIR,
    "data",
    "proj",
)
display(INPUT_DATA_DIR)
INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data/proj')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
display(gold_standard.shape)

(998, 3)

In [11]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Projected data

In [13]:
input_file = Path(INPUT_DATA_DIR, "lincs-projection.pkl").resolve()

display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data/proj/lincs-projection.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
display(lincs_projection.shape)

(987, 1170)

In [16]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
LV1,0.036115,-0.358610,0.091067,0.008923,0.046469,0.136747,0.036151,0.034024,-0.027445,0.060644,...,0.069930,0.021537,0.020511,-0.006250,-0.002660,-0.017210,-0.004029,-0.145490,0.102504,0.057621
LV2,0.012281,0.009738,0.004650,-0.006554,0.021758,0.010425,0.000794,-0.000603,0.011534,-0.004425,...,0.000344,0.001123,-0.012675,-0.004849,0.000375,0.002623,-0.003252,-0.066209,0.025378,0.012453
LV3,-0.005533,-0.059174,0.013454,0.008906,-0.004939,0.033663,-0.010045,0.003824,0.009814,-0.007492,...,-0.021941,-0.002417,0.000155,0.002941,-0.015100,-0.016372,0.012249,0.009216,0.018254,-0.003516
LV4,-0.004151,-0.032884,0.005934,-0.003991,-0.028524,0.002709,0.007001,0.024704,0.005058,-0.004345,...,0.013806,0.003197,0.013099,0.002643,-0.008836,0.023852,0.028957,0.014681,0.000826,-0.009739
LV5,-0.015156,-0.005276,0.025747,0.000346,-0.013932,-0.010911,0.041403,-0.018285,0.003507,-0.007621,...,-0.011577,-0.015956,-0.027614,-0.001707,-0.000542,-0.016918,0.001959,0.024147,0.013004,-0.014166


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f
    for f in INPUT_DATA_DIR.glob("*.pkl")
    if f.name.startswith(("smultixcan-", "spredixcan-"))
]

In [19]:
display(len(phenomexcan_input_file_list))

51

# Predict drug-disease associations

In [20]:
from drug_disease import (
    predict_dotprod,
    predict_dotprod_neg,
    predict_pearson,
    predict_pearson_neg,
    predict_spearman,
    predict_spearman_neg,
)

In [21]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)
    print(f"  shape: {phenomexcan_projection.shape}")

    predict_dotprod(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    predict_pearson(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    predict_pearson_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    predict_spearman(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    predict_spearman_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    print("\n")

spredixcan-mashr-zscores-Esophagus_Muscularis-projection.pkl
  shape: (987, 4091)
  predicting... already run
  predicting... already run
  predicting...


    shape: (1170, 4091)


    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.006884
std,0.030827
min,-0.137300
25%,-0.013479
50%,0.007564
75%,0.027909
max,0.140537


    saving to: /home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/predictions/pearson/spredixcan-mashr-zscores-Esophagus_Muscularis-projection-prediction_scores.h5


...

  predicting...


    shape: (1170, 4091)


    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,1.007517
std,0.031041
min,0.856779
25%,0.986929
50%,1.008105
75%,1.028662
max,1.147540


    saving to: /home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/predictions/spearman_neg/spredixcan-mashr-zscores-Spleen-projection-prediction_scores.h5
